In [1]:
from collections import Counter, defaultdict
from functools import partial
import numpy as np
import pandas as pd
from pprint import PrettyPrinter
from tqdm import tqdm
from z3 import *

pp = PrettyPrinter().pprint

In [2]:
## This cell is an example of the variables
## required in the data.

# persons is a list of ids (for simplicity)
# we can be more sophisticated and use a 
# Person class to represent each person
persons = range(50)


# tables is a list of table sizes
# e.g. if tables = [3, 4, 5], it means
# there are 3 tables of sizes 3, 4 and 5
# respectively.
tables = [10] * 5

# groups is a dictionary that maps a group id
# to a list of all the people in that group
# e.g. if groups[0] == [1, 2], it means
# person 1 and 2 belong to group 0
groups = defaultdict(list)
groups[0] = [0, 1, 2, 3]
groups[1] = [6, 7, 8, 9]

# friends is a dictionary that maps a person_id
# to a list of all that person's friends.
# e.g. if friends[0] == [1, 2, 3], it means 
# that person 0 has three friends: persons 1, 2, 3
friends = defaultdict(list)
friends[0] = [1]
friends[1] = [2, 3, 4]

# enemies is a list of pairs where
# each pair holds two people that 
# shouldn't be seated on the same table
enemies = []

# couples is a list of pairs where
# each pair holds two people that
# must be seated on the same table
couples = []

In [3]:
df = pd.read_csv('example_data/120_attendees.csv')

In [4]:
data_dict = dict()
for col in df.columns:
    data_dict[col] = list(filter(lambda x: not(math.isnan(x)), df[col]))
    
persons = data_dict['attendees_id']
tables = list(map(int, data_dict['table_sizes']))

get_person_index = lambda x: persons.index(x)
enemy_a_indices = map(get_person_index, data_dict['enemy_a'])
enemy_b_indices = map(get_person_index, data_dict['enemy_b'])
couple_a_indices = map(get_person_index, data_dict['must_be_together_a'])
couple_b_indices = map(get_person_index, data_dict['must_be_together_b'])
enemies = list(zip(enemy_a_indices, enemy_b_indices))
couples = list(zip(couple_a_indices, couple_b_indices))
group_member_indices = list(map(get_person_index, data_dict['group_member_id']))

groups = defaultdict(list)
for group_id, group_member_id in zip(data_dict['group_id'], group_member_indices):
    groups[group_id].append(group_member_id)

friends_set = defaultdict(set)
for _, g_members in groups.items():
    for member in g_members:
        friends_set[member] = friends_set[member].union(g_members)

friends = defaultdict(list)
for p, p_friends in friends_set.items():
    friends[p] = list(p_friends.difference(set([p])))

In [5]:
person_ids = range(len(persons))
table_ids = range(len(tables))

In [6]:
def get_assignment_booleans():
    '''
    get_assignment_booleans returns a 2d list assg,
    where assg[i][j] represents the boolean that
    will be true if person i sits on table j. And false
    if person i does not sit on table j.
    '''
    
    def get_person_table_boolean(person_id, table_id):
        return Bool('{}_{}'.format(person_id, table_id))
    
    def get_person_assignments(person_id):
        return [get_person_table_boolean(person_id, table_id) for table_id in table_ids]
        
    return [get_person_assignments(person_id) for person_id in person_ids]
    
assg = get_assignment_booleans()

In [7]:
def get_filtered_assignments(p_ids=person_ids, t_ids=table_ids):
    '''
    get_filtered_assignments takes a list of persons `p_ids`
    and a list of tables `t_ids` and returns a list of booleans
    that correspond to the persons and tables provided. that is,
    it returns a cartesian product of the two lists.
    
    if no args are provided, p_ids and t_ids default to the all
    the persons and all the tables.
    
    for e.g. if p_ids == [10], then it returns all the booleans
    that correspond to person_id == 10, which are all the booleans
    for person 0 and all the tables that person #10 can possibly 
    sit at. hence it returns a boolean list: 
    [10_0, 10_1, 10_2, ..., 10_n] where n is the last table_id.
    
    another example: if p_ids == [2, 5] and t_ids == [3, 4], then
    the resulting list of booleans should be [2_3, 2_4, 5_3, 5_4].
    hence, all the booleans returned are the cartesian product of 
    the p_ids and t_ids provided.
    
    if p_ids and t_ids are both not provided, i.e. they both
    have the default value of all persons and all tables, then
    all possible booleans are returned. i.e.
    [0_0, 0_1, ..., 0_n, 1_0, 1_1, ..., 1_n, m_0, m_1, ..., m_n]
    where m is the last person_id and n is table_id.
    '''
    f_assgs = [assg[p][t] for p in p_ids for t in t_ids]
    return f_assgs

In [8]:
def get_assignment_results(model):
    '''
    get_assignment_results takes in a z3 model
    and returns a 2d list: results where 
    results[i][j] is True if person i sits
    at table j. And False if person i does
    not sit at table j.
    
    this result matrix is a realization of the 
    booleans that the solver solves.
    '''
    if not model:
        return [[]]
    
    results = [[is_true(model.eval(assg[p_i][t_i])) for t_i in table_ids] for p_i in person_ids]
    return results

def get_couple_constraint(p1_id, p2_id):
    c1a = get_filtered_assignments(p_ids=[p1_id])
    c2a = get_filtered_assignments(p_ids=[p2_id])
    return Or(*map(And, (zip(c1a, c2a))))

def get_enemy_constraint(p1_id, p2_id):
    return Not(get_couple_constraint(p1_id, p2_id))

In [9]:
hard_cons = []

# each person can only be in one table
for p_i, _ in enumerate(persons):
    p_assg = get_filtered_assignments(p_ids=[p_i])
    cons = Sum(map(lambda p: If(p, 1, 0), p_assg)) == 1
    hard_cons.append(cons)
    
# each table has a fixed capacity
for t_i, t_cap in enumerate(tables):
    t_assg = get_filtered_assignments(t_ids=[t_i])
    cons = Sum(map(lambda p: If(p, 1, 0), t_assg)) <= t_cap
    hard_cons.append(cons)

# couples must be seated together
for c1, c2 in couples:
    c_cons = get_couple_constraint(c1, c2)
    hard_cons.append(c_cons)

# enemies must be seated separately
for e1, e2 in enemies:
    e_cons = get_enemy_constraint(e1, e2)
    hard_cons.append(e_cons)

for p, p_friends in tqdm(friends.items()):
    continue
    if len(p_friends) > 1:
        for t in table_ids:
            friend_bools = [assg[f][t] for f in p_friends]
            ge_2_friends = Sum(map(lambda p: If(p, 1, 0), friend_bools)) >= 2
            hard_cons.append(Implies(assg[p][t], ge_2_friends))
    elif len(p_friends) > 0:
        c_cons = get_couple_constraint(p, p_friends[0])
        hard_cons.append(c_cons)

# # each table should not have more than 70% from
# # any single group
for t, t_cap in tqdm(list(enumerate(tables))):
    continue
    group_cap = int(t_cap * 0.7)
    affected_groups = { k: v for k, v in groups.items() if len(v) > group_cap }
    for g, g_members in affected_groups.items():
        g_assg = get_filtered_assignments(p_ids=g_members, t_ids=[t])
        cons = Sum(map(lambda p: If(p, 1, 0), g_assg)) <= group_cap
        hard_cons.append(cons)
    
print ('constraints enumerated')

100%|██████████| 11/11 [00:00<00:00, 3903.33it/s]

constraints enumerated


In [10]:
s = Solver()
s.add(And(hard_cons))
%time check_res = s.check()
if check_res == sat:
    print ('problem solved')
    results = get_assignment_results(s.model())
    # print(pd.DataFrame(results, columns=['table_{}'.format(t) for t in table_ids]))
    ppl_assg = [r.index(True) for r in results]
    # print(pd.DataFrame(ppl_assg, columns=['table_id']))
    # print(Counter(ppl_assg))
else:
    print('not satisfiable')

CPU times: user 3.53 s, sys: 29.2 ms, total: 3.56 s
Wall time: 3.62 s
problem solved
